In [1]:
import cl_graph_bert as cgm
import torch
from torch import nn
import json
from transformers import BertModel, BertConfig, BertTokenizer, AdamW
import tqdm
import numpy as np

from torch.utils.data import Dataset, DataLoader

import dgl
g = dgl.load_graphs("./graphs/industrial_and_scientific_5_core_new.dgl")[0][0]

In [10]:
model = cgm.CLIPGraphModel(
    rel_types = g.etypes,
    emb_types = {x: g.number_of_nodes(x) for x in g.ntypes} 
)

device = "cuda"

model.load_state_dict(torch.load("./base_statedict_3.1703879752973503.pt", map_location=torch.device('cuda')))
model.eval()


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


CLIPGraphModel(
  (graph_model): StochasticTwoLayerRGCN(
    (embed): HeteroEmbedding(
      (embeds): ModuleDict(
        (Brand): Embedding(1900, 512)
        (Customer): Embedding(11041, 512)
        (Product): Embedding(5334, 512)
        (Review): Embedding(77060, 512)
      )
    )
    (conv1): HeteroGraphConv(
      (mods): ModuleDict(
        (rev_SOLD_BY): GraphConv(in=512, out=256, normalization=both, activation=None)
        (WROTE): GraphConv(in=512, out=256, normalization=both, activation=None)
        (SOLD_BY): GraphConv(in=512, out=256, normalization=both, activation=None)
        (rev_REVIEW_OF): GraphConv(in=512, out=256, normalization=both, activation=None)
        (REVIEW_OF): GraphConv(in=512, out=256, normalization=both, activation=None)
        (rev_WROTE): GraphConv(in=512, out=256, normalization=both, activation=None)
      )
    )
    (conv2): HeteroGraphConv(
      (mods): ModuleDict(
        (rev_SOLD_BY): GraphConv(in=256, out=256, normalization=both, activ

In [11]:
g

Graph(num_nodes={'Brand': 1900, 'Customer': 11041, 'Product': 5334, 'Review': 77060},
      num_edges={('Brand', 'rev_SOLD_BY', 'Product'): 5555, ('Customer', 'WROTE', 'Review'): 77060, ('Product', 'SOLD_BY', 'Brand'): 5555, ('Product', 'rev_REVIEW_OF', 'Review'): 77060, ('Review', 'REVIEW_OF', 'Product'): 77060, ('Review', 'rev_WROTE', 'Customer'): 77060},
      metagraph=[('Brand', 'Product', 'rev_SOLD_BY'), ('Product', 'Brand', 'SOLD_BY'), ('Product', 'Review', 'rev_REVIEW_OF'), ('Customer', 'Review', 'WROTE'), ('Review', 'Product', 'REVIEW_OF'), ('Review', 'Customer', 'rev_WROTE')])

In [21]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np 
import tqdm

bert_base = BertModel.from_pretrained('bert-base-uncased')
device = "cuda"

bert_base.to(device)
model.language_model.to(device)
model.graph_model.to(device)
g.to(device)

reivew_ids = []

gnn_sims = []
bert_base_sims = []
bert_finetuned_sims = []

with torch.no_grad():

    for i in tqdm.tqdm(range(g.num_nodes('Customer'))):
        review_id = g.successors(i, 'WROTE').to(device)
        
        if len(review_id) > 1 and len(review_id) < 650:

            reivew_ids.append(review_id)
            
            
            gnn_embs = model.graph_model.embed({'Review': review_id})['Review'].detach().cpu().numpy()

            
            base_bert_embs = bert_base(input_ids = g.nodes["Review"].data['input_ids'][review_id].to(device), 
               attention_mask=g.nodes["Review"].data['attention_mask'][review_id].to(device), 
               token_type_ids=g.nodes["Review"].data['token_type_ids'][review_id].to(device)).last_hidden_state[:,0].type(torch.float64).detach().cpu().numpy()
            
            bert_finetuned_embs = model.language_model(input_ids = g.nodes["Review"].data['input_ids'][review_id].to(device), 
               attention_mask=g.nodes["Review"].data['attention_mask'][review_id].to(device), 
               token_type_ids=g.nodes["Review"].data['token_type_ids'][review_id].to(device)).last_hidden_state[:,0].type(torch.float64).detach().cpu().numpy()

            gnn_sims.append( (np.sum(cosine_similarity(gnn_embs)) - len(review_id)) /  (len(review_id) ** 2 - len(review_id)) )
            bert_base_sims.append( (np.sum(cosine_similarity(base_bert_embs)) - len(review_id)) /  (len(review_id) ** 2 - len(review_id)) )
            bert_finetuned_sims.append( (np.sum(cosine_similarity(bert_finetuned_embs)) - len(review_id)) / (len(review_id) ** 2 - len(review_id)) )


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|███████████████████████████████████████████████████████| 11041/11041 [15:22<00:00, 11.97it/s]


In [12]:
## Other review sim
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np 
import tqdm

bert_base = BertModel.from_pretrained('bert-base-uncased')
device = "cuda"

bert_base.to(device)
model.language_model.to(device)
model.graph_model.to(device)
g.to(device)


reivew_ids = []

gnn_group_sims = []
bert_group_base_sims = []
bert_group_finetuned_sims = []


gnn_batch_embs = []
bert_base_embs = []
bert_finetuned_embs = []




with torch.no_grad():
    for i in tqdm.tqdm(range(0, g.num_nodes('Product'), 50)):
        for j in range(5):
            review_id = g.successors(i + j, 'rev_REVIEW_OF').to(device)

            if len(review_id) > 1 and len(review_id) < 650:

                reivew_ids.append(review_id)
                gnn_batch_embs.append(model.graph_model.embed({'Review': review_id})['Review'].detach().cpu().numpy())


                bert_base_embs.append(bert_base(input_ids = g.nodes["Review"].data['input_ids'][review_id].to(device), 
                   attention_mask=g.nodes["Review"].data['attention_mask'][review_id].to(device), 
                   token_type_ids=g.nodes["Review"].data['token_type_ids'][review_id].to(device)).last_hidden_state[:,0].type(torch.float64).detach().cpu().numpy())

                bert_finetuned_embs.append(model.language_model(input_ids = g.nodes["Review"].data['input_ids'][review_id].to(device), 
                   attention_mask=g.nodes["Review"].data['attention_mask'][review_id].to(device), 
                   token_type_ids=g.nodes["Review"].data['token_type_ids'][review_id].to(device)).last_hidden_state[:,0].type(torch.float64).detach().cpu().numpy())

gnn_group_sims.append( (np.sum(cosine_similarity(gnn_batch_embs)) - len(gnn_batch_embs) ) /  (len(gnn_batch_embs) ** 2 - len(gnn_batch_embs)) )
bert_group_base_sims.append( (np.sum(cosine_similarity(bert_base_embs)) - len(gnn_batch_embs)) /  (len(gnn_batch_embs) ** 2 - len(gnn_batch_embs)) )
bert_group_finetuned_sims.append( (np.sum(cosine_similarity(bert_finetuned_embs)) - len(gnn_batch_embs)) / (len(gnn_batch_embs) ** 2 - len(gnn_batch_embs)) )        

print(gnn_group_sims)
print(bert_group_base_sims)
print(bert_group_finetuned_sims)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|███████████████████████████████████████████████████████████| 107/107 [02:17<00:00,  1.28s/it]
/export/scratch/petros/anaconda3/envs/defbert/lib/python

ValueError: setting an array element with a sequence.

In [19]:
gnn_batch_embs = torch.cat([torch.tensor(emb) for emb in gnn_batch_embs]).numpy()
bert_base_embs = torch.cat([torch.tensor(emb) for emb in bert_base_embs]).numpy()
bert_finetuned_embs = torch.cat([torch.tensor(emb) for emb in bert_finetuned_embs]).numpy()

gnn_group_sims.append( (np.sum(cosine_similarity(gnn_batch_embs)) - len(gnn_batch_embs) ) /  (len(gnn_batch_embs) ** 2 - len(gnn_batch_embs)) )
bert_group_base_sims.append( (np.sum(cosine_similarity(bert_base_embs)) - len(gnn_batch_embs)) /  (len(gnn_batch_embs) ** 2 - len(gnn_batch_embs)) )
bert_group_finetuned_sims.append( (np.sum(cosine_similarity(bert_finetuned_embs)) - len(gnn_batch_embs)) / (len(gnn_batch_embs) ** 2 - len(gnn_batch_embs)) )        

print(gnn_group_sims)
print(bert_group_base_sims)
print(bert_group_finetuned_sims)

[5.8056439452622435e-06]
[0.7796098610781403]
[0.9467936051933957]


In [15]:
print("Average bert base", np.mean(bert_group_base_sims))
print("Average bert fine-tuned", np.mean(bert_group_finetuned_sims))

Average bert base 0.9443831244999533
Average bert fine-tuned 0.982382160829002


In [4]:
for i in range(len(bert_base_sims)):
    print(gnn_sims[i], bert_base_sims[i], bert_finetuned_sims[i])

0.004453218900240385 0.8268821558012124 0.9641568140789943
-0.002916847988640591 0.7837415820510998 0.958108478312223
-0.0008572816132783173 0.7653988364540306 0.9572925094031238
0.002684132258097331 0.8021195736030341 0.9537909455563783
-0.015741825103759766 0.8181333275989203 0.9549214563720874
0.013798666000366212 0.8423244155791278 0.9486003139416198
0.017736244201660156 0.7435472558954691 0.9623192758146976
-0.011234784126281738 0.7791316478116391 0.9530580884998683
0.008894012087867373 0.7784917828060489 0.9556224870825016
0.004416480208888199 0.7968453673194399 0.9551825420109938
0.0027085807588365343 0.7949908159003145 0.9470703084905929
0.002252124223902374 0.7728560316364009 0.9526907641745624
0.0019639153634348223 0.7944981703043286 0.954046516304083
0.0014678107367621528 0.7671140535778513 0.9451950183476456
0.0003506527152112735 0.8116810987891644 0.9578567358675356
0.0005263328552246094 0.7687704494871236 0.9567928327612203
0.0005958781522863052 0.7891434623546072 0.96101

-0.002414989471435547 0.8272577310388576 0.9539747865957281
0.009176301956176757 0.7750252120097532 0.9604159080494272
0.0053806304931640625 0.7964277821185805 0.9549412254246975
0.009444379806518554 0.742283944237304 0.9550538626330062
0.0028958870814396786 0.8108913727233099 0.9642516742048971
0.0019794464111328124 0.7761247314280638 0.9578985360842832
-0.004639060129761225 0.7811875644771713 0.9594244646345969
-0.021369147300720214 0.7830765781418709 0.9493208999608627
0.0016975743430001394 0.7462116182580852 0.9626959078113949
-0.0089527342054579 0.8309669525137404 0.9560407232051934
0.019317483901977538 0.8302150023704967 0.960632206265048
-0.00010849879338191106 0.7838950154367927 0.9585126371911733
0.018215395155407134 0.7917780515886145 0.9585483951760477
0.010423914591471354 0.7867296314272226 0.951233941486818
0.007795635859171549 0.8357864022243271 0.9706337276796307
0.019272589683532716 0.8022170951169535 0.9642279301923384
-0.0015494462215539181 0.8102464219493469 0.960761

0.00756219105842786 0.7858311976643418 0.9671756594916529
0.004135105344984267 0.8010731740510175 0.9661063809299508
-0.0016109657287597657 0.7822063305929824 0.9604027990937671
-0.0010583837469060858 0.828650258224927 0.962596668362763
0.0021700329250759548 0.8393625742372234 0.9595142086515571
-0.0117722749710083 0.8122554091397445 0.9735289497753928
-0.008980376379830497 0.7321493700932542 0.9580489331929065
0.007473094122750419 0.7825151895327436 0.9564790869015096
-0.001252905527750651 0.7744011534350169 0.9570650630317481
0.014080142974853516 0.8139499317034998 0.9485028682437658
-0.008446192741394043 0.8648208834648393 0.9646119294246507
-0.004076175200633514 0.7618582150110996 0.9640377259702265
0.001804563734266493 0.8047161613161625 0.9594600870409892
-0.02457008361816406 0.8014632027976957 0.9556663997248098
0.006203095118204753 0.8527073696024556 0.9553391826920451
-0.007541196686880929 0.751446148128272 0.9659635183362812
0.0011496861775716146 0.739543980763356 0.949745662

-0.019133361180623372 0.8768172964892218 0.9557736805167711
-0.00619702868991428 0.7910989959496715 0.9505334675135505
-0.005530397097269694 0.7987906993877056 0.9557011125099031
-0.0025437617162514846 0.8303092883288711 0.9816425113228573
0.006231843379505894 0.8603349416990363 0.9850417341134676
-0.004174217101066343 0.7714987562293785 0.9511774670742054
-0.007463333823464134 0.7898459933828932 0.9585645431777923
0.003601672157408699 0.7880578845540681 0.9584796674524427
0.002417255583263579 0.7976680269622664 0.9606672882513687
0.009427070617675781 0.8307313315974186 0.9719170746733425
-0.0098460939195421 0.8262453309652119 0.9619421512412433
0.003281610662286932 0.7804414987402892 0.9581170291731411
-0.0007995764414469401 0.8055118018580644 0.9685210774932848
0.022222423553466798 0.7870509110149975 0.9548062348538743
-0.013190831456865584 0.8013039095898323 0.9640856653747228
-0.0004941133352426382 0.8216407188064286 0.9549303493379875
-0.0047896385192871095 0.8443174443161638 0.96

In [22]:
print("Average bert base", np.mean(bert_base_sims))
print("Average bert fine-tuned", np.mean(bert_finetuned_sims))
print("Average difference", np.mean(np.array(bert_finetuned_sims) - np.array(bert_base_sims)))


Average bert base 0.8549195565506538
Average bert fine-tuned 0.9586276618802808
Average difference 0.103708105329627


In [10]:
print(np.mean(gnn_sims[:len(gnn_sims)//2]))
print(np.mean(gnn_sims[len(gnn_sims)//2:]))

0.5007986689840875
0.02319790494159576


In [7]:
model.to('cpu')
model.graph_model.embed({'Review': torch.tensor([1])})['Review'].detach().cpu().numpy()

array([[-8.37563217e-01, -1.62651047e-01, -5.99956453e-01,
         1.03902018e+00,  4.68235850e-01, -2.66605914e-01,
         9.35316324e-01,  1.70611656e+00, -2.79763013e-01,
         7.21681833e-01,  1.40554607e-01,  3.33048910e-01,
        -4.87642050e-01,  8.95017803e-01,  2.29110926e-01,
        -1.20437336e+00,  2.09761053e-01, -1.16526234e+00,
        -8.18220079e-01,  4.28669959e-01, -2.94622332e-01,
        -3.40034992e-01, -5.06422162e-01, -1.17381310e+00,
         5.69969058e-01, -5.03100574e-01,  2.03424320e-01,
         4.51832473e-01,  6.48726523e-01,  2.32269600e-01,
        -1.61208436e-01,  4.08931196e-01,  1.67983081e-02,
        -5.56555152e-01,  4.19612855e-01, -1.76771760e+00,
        -4.01486233e-02,  4.51788485e-01, -1.56105852e+00,
        -1.76364541e-01,  4.09409732e-01, -7.07604587e-01,
         9.85402584e-01,  5.37448004e-02, -4.53736007e-01,
         1.10507703e+00,  1.86479345e-01,  6.54095262e-02,
         1.15016364e-01,  3.18524390e-01, -7.54547775e-0

In [13]:
np.mean(gnn_embs, axis=0).shape

(512,)

In [16]:
np.mean(base_bert_embs, axis=0).shape

(768,)

In [2]:
g

Graph(num_nodes={'Brand': 1900, 'Customer': 11041, 'Product': 5334, 'Review': 77060},
      num_edges={('Brand', 'rev_SOLD_BY', 'Product'): 5555, ('Customer', 'WROTE', 'Review'): 77060, ('Product', 'SOLD_BY', 'Brand'): 5555, ('Product', 'rev_REVIEW_OF', 'Review'): 77060, ('Review', 'REVIEW_OF', 'Product'): 77060, ('Review', 'rev_WROTE', 'Customer'): 77060},
      metagraph=[('Brand', 'Product', 'rev_SOLD_BY'), ('Product', 'Brand', 'SOLD_BY'), ('Product', 'Review', 'rev_REVIEW_OF'), ('Customer', 'Review', 'WROTE'), ('Review', 'Product', 'REVIEW_OF'), ('Review', 'Customer', 'rev_WROTE')])

In [6]:
customer_reviews = []
for i in tqdm.tqdm(range(g.num_nodes('Customer'))):
    review_id = g.successors(i, 'WROTE')
    customer_reviews.append(len(review_id))
print(np.mean(customer_reviews))


product_reviews = []

for i in tqdm.tqdm(range(0, g.num_nodes('Product'), 50)):
    review_id = g.successors(i, 'rev_REVIEW_OF')
    product_reviews.append(len(review_id))
print(np.mean(product_reviews))


100%|████████████████████████████████████████████████████| 11041/11041 [00:00<00:00, 13802.68it/s]


6.9794402680916585


100%|████████████████████████████████████████████████████████| 107/107 [00:00<00:00, 11221.73it/s]

12.63551401869159


In [7]:
15/10

1.5

In [8]:
12.63551401869159/ 6.9794402680916585

1.8103907381309867